In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np 
import os
import canaro
import cv2 as cv


In [4]:
import caer
import gc

In [9]:
IMG_SIZE=(80,80)
channels=1
char_path=r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [10]:
char_dict={}
for char in os.listdir(char_path):
    char_dict[char]=len(os.listdir(os.path.join(char_path,char)))
    

In [11]:
# sort dict
char_dict=caer.sort_dict(char_dict,descending=True)
char_dict

In [13]:
characters=[]
count=0
for i in char_dict:
    characters.append(i[0])
    count+=1
    if count>=10:
        break

In [14]:
# create training data
train=caer.preprocess_from_dir(char_path,characters,channels=channels,IMG_SIZE=IMG_SIZE,isShuffle=True)

In [15]:
len(train)

In [18]:
import matplotlib.pyplot as plt

In [21]:
plt.figure(figsize=(30,30))
plt.imshow(train[0][0],cmap='gray')
plt.show()


In [22]:
featureSet,labels=caer.sep_train(train,IMG_SIZE=IMG_SIZE)

In [23]:
# NORMALISED THE FEATURE SET TO FAST PROCESS
from tensorflow.keras.utils import to_categorical
featureSet=caer.normalize(featureSet)
labels=to_categorical(labels,len(characters))

In [24]:
x_train,x_val,y_train,y_val=caer.train_val_split(featureSet,labels,val_ratio=.2)

In [25]:
del train
del featureSet
del labels
gc.collect()

In [26]:
BATCH_SIZE=32
EPOCHS=10

In [27]:
# img data generator
datagen=canaro.generators.imageDataGenerator()
train_gen=datagen.flow(x_train,y_train,batch_size=BATCH_SIZE)

In [30]:
# CREATE MODEL
model=canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE,channels=channels,output_dim=len(characters),loss='binary_crossentropy',decay=1e-6,learning_rate=0.001,momentum=0.9,nesterov=True)

In [31]:
model.summary()

In [32]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list=[LearningRateScheduler(canaro.lr_schedule)]

In [33]:
training=model.fit(train_gen,steps_per_epoch=len(x_train)//BATCH_SIZE,epochs=EPOCHS,validation_data=(x_val,y_val),validation_steps=len(y_val)//BATCH_SIZE,callbacks=callbacks_list)

In [34]:
test_path=r'../input/the-simpsons-characters-dataset/simpsons_dataset/bart_simpson/pic_0015.jpg'
img=cv.imread(test_path)
plt.imshow(img,cmap='gray')
plt.show()

In [39]:
def prepare(img):
    img=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    img=cv.resize(img,IMG_SIZE)
    img=caer.reshape(img,IMG_SIZE,1)
    return img

In [40]:
predictions=model.predict(prepare(img))

In [41]:
print(characters[np.argmax(predictions[0])])